**<hr><h>Capstone Project: Battle of Texas city living for young adults<h><hr>**

**Project Objective:**

The Objective of this project is to determine the best city that provides conveniences for young adults in Texas. The major cities being considered are Austin, Dallas, Houston and San Antonio. The  criteria to be considered are venues for restaurants, activities like yoga, parks accessibility for biking.  |

**Methodology:**

The data to be used is the coordinates from the center of downtown and the venues to be considered are data from Foursquare. A 3.1 mile (5Km) radius from the center of downtown will be considered for the analysis. The clusters will look at type, location ratings and accessibility of these conveniences being considered for young adults.
A report will analyze using metrics and plots to compare and discuss pro and cons of these four cities being considered.

**Input Coordinates table of TX cities into Pandas**

In [1]:
import pandas as pd
df = pd.read_csv("TXcities.csv")
df.head()

,TX_City,Latitude,Longitude
0,Austin,30.267153,-97.743057
1,Downtown Houston,29.751869,-95.327438
2,Downtown Dallas,32.776665,-96.796989
3,Downtown San Antonio,29.424122,-98.493629


**Setup Libraries for Data Analysis and Visualization**

In [2]:
! pip install --upgrade pip
! pip install beautifulsoup4
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.request
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda update -n base -c defaults conda --yes
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 1.4MB 30.7MB/s eta 0:00:01
  Found existing installation: pip 19.1.1
    Uninstalling pip-19.1.1:
      Successfully uninstalled pip-19.1.1
     |████████████████████████████████| 102kB 17.4MB/s ta 0:00:01
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          

**Import Foursquare Credentials**

In [3]:
CLIENT_ID = '55ELXDS21BUOO4A2LBRM4T35UEOVCOFUYKG0KXTIPUI5OIWU' # your Foursquare ID
CLIENT_SECRET = 'KQO5CBRZRY1IDSUGQZOAPU0RR5QDG5A55DZAW4FOMSTPARLJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 55ELXDS21BUOO4A2LBRM4T35UEOVCOFUYKG0KXTIPUI5OIWU
CLIENT_SECRET:KQO5CBRZRY1IDSUGQZOAPU0RR5QDG5A55DZAW4FOMSTPARLJ


In [5]:
address = 'Austin, TX'

geolocator = Nominatim(user_agent="tx_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Austin, TX are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Austin, TX are 30.2711286, -97.7436995.


**Create map of four TX cities for analysis**

In [6]:
# create map of Tx Cities using latitude and longitude values
map_tx = folium.Map(location=[latitude, longitude], zoom_start=6.49)

# add markers to map
for lat, lng, TX_city in zip(df['Latitude'], df['Longitude'],  df['TX_City']):
   #label = '{}, {}'.format(Neighborhood, borough)
   # label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tx)  
    
map_tx

In [7]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'TX_City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                              neighborhood_longitude))

Latitude and longitude values of Austin are 30.267153000000004, -97.743057.


**Create URL of venues within 3.1 Miles of Austin center**

In [8]:
#Double-click __here__ for the solution.
#<!-- The correct answer is:
LIMIT = 200 # limit of number of venues returned by Foursquare API
#-->

#<!--
radius = 5000 # define radius
#-->

#<!--
#\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
#--> 

'https://api.foursquare.com/v2/venues/explore?&client_id=55ELXDS21BUOO4A2LBRM4T35UEOVCOFUYKG0KXTIPUI5OIWU&client_secret=KQO5CBRZRY1IDSUGQZOAPU0RR5QDG5A55DZAW4FOMSTPARLJ&v=20180604&ll=30.267153000000004,-97.743057&radius=5000&limit=200'

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
nbh_venues = getNearbyVenues(names=df['TX_City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Austin
Downtown Houston
Downtown Dallas
Downtown San Antonio


In [14]:
print(nbh_venues.shape)
nbh_venues.head()


(400, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Austin,30.267153,-97.743057,JuiceLand,30.266037,-97.742623,Juice Bar
1,Austin,30.267153,-97.743057,The Roosevelt Room,30.267842,-97.746242,Bar
2,Austin,30.267153,-97.743057,Houndstooth Coffee,30.266194,-97.743025,Coffee Shop
3,Austin,30.267153,-97.743057,Eddie V's Prime Seafood,30.266339,-97.740504,Seafood Restaurant
4,Austin,30.267153,-97.743057,Alamo Drafthouse Cinema,30.267460,-97.739550,Movie Theater


In [15]:
nbh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Austin,100,100,100,100,100,100
Downtown Dallas,100,100,100,100,100,100
Downtown Houston,100,100,100,100,100,100
Downtown San Antonio,100,100,100,100,100,100


In [16]:
print('There are {} uniques categories.'.format(len(nbh_venues['Venue Category'].unique())))

There are 124 uniques categories.


In [19]:
# one hot encoding
nbh_onehot = pd.get_dummies(nbh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nbh_onehot['Neighborhood'] = nbh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nbh_onehot.columns[-1]] + list(nbh_onehot.columns[:-1])
nbh_onehot = nbh_onehot[fixed_columns]

nbh_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Burger Joint,Café,Cajun / Creole Restaurant,Capitol Building,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Dog Run,Farmers Market,Field,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gaming Cafe,Garden Center,Gas Station,Gastropub,General College & University,German Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Piano Bar,Pizza Place,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Snack Place,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Austin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Austin,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Austin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Austin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Austin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
nbh_grouped = nbh_onehot.groupby('Neighborhood').mean().reset_index()
nbh_grouped.shape

(4, 125)

**Top 20 Venues by City (Neighborhood)**

In [52]:
num_top_venues = 20
for hood in nbh_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nbh_grouped[nbh_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Austin----
                    venue  freq
0                   Hotel  0.06
1              Food Truck  0.05
2             Coffee Shop  0.05
3             Yoga Studio  0.04
4            Cocktail Bar  0.03
5      Seafood Restaurant  0.03
6              Restaurant  0.03
7           Grocery Store  0.03
8          Ice Cream Shop  0.03
9                     Bar  0.03
10    American Restaurant  0.02
11                    Gym  0.02
12               Dive Bar  0.02
13         Cosmetics Shop  0.02
14    Sporting Goods Shop  0.02
15  Performing Arts Venue  0.02
16             Taco Place  0.02
17     Italian Restaurant  0.02
18              BBQ Joint  0.02
19           Burger Joint  0.02


----Downtown Dallas----
                              venue  freq
0                        Steakhouse  0.07
1                       Coffee Shop  0.07
2                             Hotel  0.06
3                               Bar  0.05
4               American Restaurant  0.04
5               Japanese Restaurant

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nbh_grouped['Neighborhood']

for ind in np.arange(nbh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nbh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Austin,Hotel,Food Truck,Coffee Shop,Yoga Studio,Ice Cream Shop,Seafood Restaurant,Cocktail Bar,Bar,Restaurant,Grocery Store
1,Downtown Dallas,Steakhouse,Coffee Shop,Hotel,Bar,American Restaurant,Park,Japanese Restaurant,Pizza Place,Burger Joint,Cocktail Bar
2,Downtown Houston,Bar,Mexican Restaurant,Hotel,Music Venue,Beer Garden,Brewery,Vietnamese Restaurant,New American Restaurant,Italian Restaurant,Baseball Stadium
3,Downtown San Antonio,Hotel,American Restaurant,Plaza,Theater,Mexican Restaurant,Cocktail Bar,Sandwich Place,Steakhouse,Lounge,Park


In [38]:
# set number of clusters
kclusters = 4

nbh_grouped_clustering = nbh_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nbh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Austin,Hotel,Food Truck,Coffee Shop,Yoga Studio,Ice Cream Shop,Seafood Restaurant,Cocktail Bar,Bar,Restaurant,Grocery Store
1,3,Downtown Dallas,Steakhouse,Coffee Shop,Hotel,Bar,American Restaurant,Park,Japanese Restaurant,Pizza Place,Burger Joint,Cocktail Bar
2,2,Downtown Houston,Bar,Mexican Restaurant,Hotel,Music Venue,Beer Garden,Brewery,Vietnamese Restaurant,New American Restaurant,Italian Restaurant,Baseball Stadium
3,1,Downtown San Antonio,Hotel,American Restaurant,Plaza,Theater,Mexican Restaurant,Cocktail Bar,Sandwich Place,Steakhouse,Lounge,Park


In [55]:
 #add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#del df_nbh['Postal Code']
df=df.rename(columns = {'TX_City':'Neighborhood'})
nbh_merger = df
from sklearn.cluster import KMeans

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nbh_merger = nbh_merger.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
indexNames = nbh_merger[nbh_merger['Cluster Labels'] == "NaN "].index #look through column to NaN
#nbh_merger=nbh_merger.dropna()
nbh_merger['Cluster Labels'] = nbh_merger['Cluster Labels'].astype(int)
nbh_merger.head(20
            ) # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Austin,30.267153,-97.743057,0,Hotel,Food Truck,Coffee Shop,Yoga Studio,Ice Cream Shop,Seafood Restaurant,Cocktail Bar,Bar,Restaurant,Grocery Store
1,Downtown Houston,29.751869,-95.327438,2,Bar,Mexican Restaurant,Hotel,Music Venue,Beer Garden,Brewery,Vietnamese Restaurant,New American Restaurant,Italian Restaurant,Baseball Stadium
2,Downtown Dallas,32.776665,-96.796989,3,Steakhouse,Coffee Shop,Hotel,Bar,American Restaurant,Park,Japanese Restaurant,Pizza Place,Burger Joint,Cocktail Bar
3,Downtown San Antonio,29.424122,-98.493629,1,Hotel,American Restaurant,Plaza,Theater,Mexican Restaurant,Cocktail Bar,Sandwich Place,Steakhouse,Lounge,Park


In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6.49)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nbh_merger['Latitude'], nbh_merger['Longitude'], nbh_merger['Neighborhood'], nbh_merger['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=.2,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Exploring Austin Top 10**

In [70]:
nbh_merger.loc[nbh_merger['Cluster Labels'] == 0, nbh_merger.columns[[0] + list(range(3, nbh_merger.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Austin,0,Hotel,Food Truck,Coffee Shop,Yoga Studio,Ice Cream Shop,Seafood Restaurant,Cocktail Bar,Bar,Restaurant,Grocery Store


**Exploring Downtown Dallas Top 10**

In [71]:
nbh_merger.loc[nbh_merger['Cluster Labels'] == 3, nbh_merger.columns[[0] + list(range(3, nbh_merger.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Dallas,3,Steakhouse,Coffee Shop,Hotel,Bar,American Restaurant,Park,Japanese Restaurant,Pizza Place,Burger Joint,Cocktail Bar


**Exploring Downtown Houston Top 10**

In [74]:
nbh_merger.loc[nbh_merger['Cluster Labels'] == 2, nbh_merger.columns[[0] + list(range(3, nbh_merger.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Houston,2,Bar,Mexican Restaurant,Hotel,Music Venue,Beer Garden,Brewery,Vietnamese Restaurant,New American Restaurant,Italian Restaurant,Baseball Stadium
